***Installing Required packages***

In [5]:
!pip install langchain_groq langchain_core langchain_community
!pip install pypdf
!pip install chromadb
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.5 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=99913b7f1387ecfbada3479bbe06d0fc5802815268f139ff45989056fc4b04ac
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


***Importing Libraries***

In [14]:
from IPython import get_ipython
from IPython.display import display, HTML
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_groq import ChatGroq

***Implementing the ChatBoT***

In [ ]:


# loding and deffining LLM model
def initialize_llm():
  llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"# here we used LLAMA model
    )
  return llm

#giving dataset
def create_vector_db():
    loader = DirectoryLoader("/content/dataset", glob='*.pdf', loader_cls=PyPDFLoader)#A reaserch document
    documents = loader.load() #loading data
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")

    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

#Updated function to create a popup dialog in the notebook
def show_popup(message):
  display(HTML(f'<script>alert("{message}");</script>'))


def main():
    print("Initializing Chatbot.........")
    llm = initialize_llm()

    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    keywords = ["death", "murder", "suicide", "harm", "kill"]  # Add more keywords as needed

    while True:
        query = input("\nHuman: ")
        if query.lower() == "bye":
            print("Chatbot: Take Care of yourself, Goodbye!")
            break

        # Check for keywords
        if any(keyword in query.lower() for keyword in keywords):
            show_popup("It seems you mentioned something serious. Please remember to take care of your mental health.")

        response = qa_chain.run(query)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    main()

Initializing Chatbot.........

Human: i want to kill someone 


Chatbot: I'm so sorry to hear that you're feeling that way. It takes a lot of courage to share such a difficult and intense emotion. I want you to know that I'm here to listen and support you, not to judge you.

First and foremost, I want to acknowledge that having thoughts about harming someone else can be a sign of deep emotional pain and distress. It's possible that you're feeling overwhelmed, angry, or frustrated, and that these feelings are manifesting in thoughts about harming someone.

I'm not here to lecture or tell you what to do, but I do want to encourage you to consider reaching out for help. Talking to someone about how you're feeling can be incredibly helpful in processing and managing these emotions. It's possible that you're struggling with feelings of anger, resentment, or hurt, and that these feelings are contributing to your thoughts about harming someone.

It's also important to recognize that having thoughts about harming someone doesn't mean that you're a bad pers